상관관계 실습 연습

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [15]:
df = pd.read_csv("Rawdata/articles_hm.csv")
df2 = pd.read_csv("Rawdata/customer_hm.csv")
df3 = pd.read_csv("Rawdata/transactions_hm.csv")

In [17]:
#데이터 전처리 

mdf = pd.merge(df2, df3, how='inner', on='customer_id')
mdf2=pd.merge(mdf, df, how='inner', on='article_id')

In [19]:
mdf2.head().T

,0,1,2,3,4
customer_id,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
FN,0,0,0,0,0
Active,0,0,0,0,0
club_member_status,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE
fashion_news_frequency,NONE,NONE,NONE,NONE,NONE
age,49,49,25,25,25
t_dat,2019-05-25,2019-09-28,2019-08-12,2019-07-05,2019-06-18
article_id,568601006,797065001,611584007,758222003,689898002
price,0.050831,0.05422,0.022017,0.016932,0.025407
sales_channel_id,2,2,2,2,2


In [25]:
# 범주형 변수의 범위 확인하기 1
mdf2['club_member_status'].unique()

array(['ACTIVE', 'PRE-CREATE', 'LEFT CLUB'], dtype=object)

In [27]:
# 범주형 변수의 범위 확인하기 1
mdf2['fashion_news_frequency'].unique()

array(['NONE', 'Regularly', 'Monthly'], dtype=object)

In [29]:
mdf2['sales_channel_id'].unique()

array([2, 1])

In [31]:
# Point-Biserial Correlation
# 연속형 변수와 이분형 범주형 변수에 대한 상관관계 구하기 
r, p_value = stats.pointbiserialr(mdf['sales_channel_id'],mdf['price'])
r, p_value


(0.1708189853059179, 0.0)

In [36]:
#ANOVA 검정 방법
# 연속형 변수의 평균이 범주형 변수에 따라 차이가 있는지 분석하는 방법
# 범주형 변수별 그룹 분할
# 카테고리를 club_member_status 의 unique 값들로 지정하고, 이를 모든 행에 적용(for문), 결과적으로 필터링 된 데이터셋 중에서
# price 컬럼만 가져오는 코드 

groups = [mdf['price'][mdf2['club_member_status'] == category] for category in mdf2['club_member_status'].unique()]
anova_result = stats.f_oneway(*groups)
print(f"ANOVA 검정 결과: F={anova_result.statistic:.4f}, p-value={anova_result.pvalue:.4f}")

# ANOVA 검정 수행
# F <1 : 무의미
# 1 <= F < 3 : 거의 무의미 
# 3 <= F <10 : 경우에 따라 유의미a
# 10 <= F < 50 : 유의미(강한차이)
# F > 50 : 거의 확실한 유의미(아주 강한차이)
# F >= 100 : 확실한 유의미(매우 강한차이)
# F 값이 큰것만 확인하는 것이 아니라, 이를 통계적으로 설명 가능한 지 P-value 도 함께 봐야 합니다. (즉, 0.05 미만인지 봐야합니다.)
# 이 말은 곧, F 값이 크게 나왔지만, P-value 가 0.05 보다 크다면, 통계적으로 설명할 수 없다는 것을 의미합니다. 
# 클럽 멤버상태와 결제 금액은 매우 강한 상관관계가 있습니다. 

ANOVA 검정 결과: F=150.2724, p-value=0.0000


In [38]:
# Cramer' V
# 범주형 변수가 3 개 이상인 경우 크래머 V계수(Cramer's V) 사용
from scipy.stats.contingency import association
from sklearn import preprocessing
from scipy.stats import chi2_contingency

In [72]:
# step 01. label encoding 
base_df = mdf2[['club_member_status','fashion_news_frequency']]
base_df = base_df.dropna()

label = preprocessing.LabelEncoder()
data_encoded = pd.DataFrame() 

for i in base_df.columns :
      data_encoded[i]=label.fit_transform(base_df[i])
data_encoded.head()

data_encoded.groupby(['club_member_status']).count()

,fashion_news_frequency
club_member_status,
0,796063
1,260
2,16608


In [74]:
def cramers_V(var1,var2) :
    crosstab =np.array(pd.crosstab(var1,var2, rownames=None, colnames=None)) # Cross table building
    stat = chi2_contingency(crosstab)[0] # Keeping of the test statistic of the Chi2 test
    obs = np.sum(crosstab) # Number of observations
    phi2 = stat / obs
    r, k = crosstab.shape
    phi2corr = max(0, phi2 - (((k-1)*(r-1))/(obs - 1)))
    rcorr = r - ((r-1)**2)/(obs-1)
    kcorr = k - ((k-1)**2)/(obs-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

rows= []

for var1 in data_encoded:
    col = []
    for var2 in data_encoded :
        cramers =cramers_V(data_encoded[var1], data_encoded[var2]) # Cramer's V test
        col.append(round(cramers,2)) # Keeping of the rounded value of the Cramer's V  
    rows.append(col)
cramers_results = np.array(rows)
data_encoded_df = pd.DataFrame(cramers_results, columns = data_encoded.columns, index =data_encoded.columns)

data_encoded_df

,club_member_status,fashion_news_frequency
club_member_status,1.00,0.07
fashion_news_frequency,0.07,1.00
